In [1]:
# Calling all the libraries that we are going to use here #
library(dplyr)
library(Boruta)
library(partykit)
library(caret)
library(party)

Warning message:
"package 'dplyr' was built under R version 3.5.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: ranger
Warning message:
"package 'partykit' was built under R version 3.5.2"Loading required package: grid
Loading required package: libcoin
Loading required package: mvtnorm
Warning message:
"package 'caret' was built under R version 3.5.2"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'party' was built under R version 3.5.2"Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Loading required package: sandwich

Attaching package: 'party'

The following objects

In [2]:
# Import DataSet #
re.imp<-read.csv("F:\\melb.realstate.all.files\\OutputDataSets\\realestate.derived.without.year.csv", header = T ,stringsAsFactors = T, sep = ',', strip.white = TRUE, na.strings = "")
re.imp<-re.imp[,-c(2,8,17,18,21)] #Excluding Address, Date, Latt, Longt & Price.bucket
str(re.imp)

'data.frame':	8220 obs. of  17 variables:
 $ Suburb      : Factor w/ 140 levels "Abbotsford","Aberfeldie",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Rooms       : int  2 2 3 3 4 2 3 2 3 2 ...
 $ Type        : Factor w/ 3 levels "h","t","u": 1 1 1 1 1 1 1 1 1 1 ...
 $ Price       : int  1480000 1035000 1465000 850000 1600000 941000 1876000 1636000 1000000 1097000 ...
 $ Method      : Factor w/ 5 levels "PI","S","SA",..: 2 2 4 1 5 2 2 2 2 2 ...
 $ SellerG     : Factor w/ 178 levels "Abercromby's",..: 18 18 18 18 111 83 111 111 83 18 ...
 $ Distance    : num  2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2.5 ...
 $ Postcode    : int  3067 3067 3067 3067 3067 3067 3067 3067 3067 3067 ...
 $ Bedroom2    : int  2 2 3 3 3 2 4 2 3 3 ...
 $ Bathroom    : int  1 1 2 2 1 1 2 1 1 1 ...
 $ Car         : int  1 0 0 1 2 0 0 2 1 2 ...
 $ Landsize    : num  202 156 134 94 120 ...
 $ BuildingArea: num  123 79 150 123 142 ...
 $ CouncilArea : Factor w/ 19 levels "Banyule","Bayside",..: 19 19 19 19 19 19 19 19 19 19 ...
 $ Day

In [3]:
# Converting Attriutes's type as Categorical # 
re.imp$Rooms<-as.factor(re.imp$Rooms)
re.imp$Type<-as.factor(re.imp$Type)
re.imp$Bedroom2<-as.factor(re.imp$Bedroom2)
re.imp$Bathroom<-as.factor(re.imp$Bathroom)
re.imp$Car<-as.factor(re.imp$Car)

In [4]:
# Filtering on basis of Type & removing Type col from them for further analysis #

## H type ##
re.imp.h<-filter(re.imp, Type %in% c("h"))
re.imp.h<-re.imp.h[,-3]

In [5]:
## T type ##
re.imp.t<-filter(re.imp, Type %in% c("t"))
re.imp.t<-re.imp.t[,-3]

In [6]:
## U Type ##
re.imp.u<-filter(re.imp, Type %in% c("u"))
re.imp.u<-re.imp.u[,-3]

In [7]:
# Applying Boruta for fnding significant variables & Random Forests to know their importance #

## Aplying Boruta on whole data set with respect to Price  ##
boruta.re.imp<-Boruta(Price~.,data = re.imp,doTrace=2)
print(boruta.re.imp)

 1. run of importance source...
 2. run of importance source...
 3. run of importance source...
 4. run of importance source...
 5. run of importance source...
 6. run of importance source...
 7. run of importance source...
 8. run of importance source...
 9. run of importance source...
 10. run of importance source...
 11. run of importance source...
After 11 iterations, +5.2 mins: 
 confirmed 14 attributes: Bathroom, Bedroom2, BuildingArea, Car, CouncilArea and 9 more;
 still have 2 attributes left.

 12. run of importance source...
 13. run of importance source...
 14. run of importance source...
 15. run of importance source...
 16. run of importance source...
 17. run of importance source...
 18. run of importance source...
 19. run of importance source...
 20. run of importance source...
 21. run of importance source...
 22. run of importance source...
 23. run of importance source...
 24. run of importance source...
 25. run of importance source...
 26. run of importance source.

Boruta performed 39 iterations in 22.19351 mins.
 15 attributes confirmed important: Bathroom, Bedroom2, BuildingArea,
Car, CouncilArea and 10 more;
 1 attributes confirmed unimportant: Months;


In [8]:
### Removing Months since it is not that important ###
re.imp<-re.imp[,-c(17)]      


In [33]:
### After Boruta, applying Cforest On whole data set ###
## Cforest to know the significance of each individual variable on price ##
re.imp.rf<-party::cforest(Price~. ,data = re.imp, control=cforest_unbiased(mtry=14,ntree=200))
impvars.re.imp<-varImp(re.imp.rf)

In [41]:
print(impvars.re.imp)


                  Overall
Suburb       352771099916
Rooms        119313034345
Type          64315921164
Method         2064104319
SellerG        4043305286
Distance      10687145199
Postcode       3377712697
Bedroom2      12588951798
Bathroom      14024669545
Car            1386244497
Landsize      41449700284
BuildingArea  10631637056
CouncilArea   31690235697
Days.diff      2714445420
Dist.bucket    3438668095


In [42]:
write.table(impvars.re.imp, file = "F:\\melb.realstate.all.files\\OutputDataSets\\ImpVarsForPriceWithoutYear.csv",row.names=T, na="",col.names=T, sep=",")


In [13]:
## Apllying Boruta for type h ##
boruta.re.imp.h<-Boruta(Price~.,data = re.imp.h,doTrace=2)
print(boruta.re.imp.h)


 1. run of importance source...
 2. run of importance source...
 3. run of importance source...
 4. run of importance source...
 5. run of importance source...
 6. run of importance source...
 7. run of importance source...
 8. run of importance source...
 9. run of importance source...
 10. run of importance source...
 11. run of importance source...
After 11 iterations, +2.7 mins: 
 confirmed 13 attributes: Bathroom, Bedroom2, BuildingArea, Car, CouncilArea and 8 more;
 still have 2 attributes left.

 12. run of importance source...
 13. run of importance source...
 14. run of importance source...
 15. run of importance source...
 16. run of importance source...
 17. run of importance source...
 18. run of importance source...
After 18 iterations, +4.4 mins: 
 rejected 1 attribute: Months;
 still have 1 attribute left.

 19. run of importance source...
 20. run of importance source...
 21. run of importance source...
 22. run of importance source...
 23. run of importance source...
 

Boruta performed 99 iterations in 21.57867 mins.
 13 attributes confirmed important: Bathroom, Bedroom2, BuildingArea,
Car, CouncilArea and 8 more;
 1 attributes confirmed unimportant: Months;
 1 tentative attributes left: Method;


In [15]:
### removing months as per Boruta it is not important ###
re.imp.h<-re.imp.h[,-c(16)] 

In [16]:
## Cforest to know the significance of each individual variable on price with Type "h" ##
re.imp.h.rf<-party::cforest(Price~. ,data = re.imp.h, control=cforest_unbiased(mtry=14,ntree=200))

In [18]:
print(re.imp.h.rf)


	 Random Forest using Conditional Inference Trees

Number of trees:  200 

Response:  Price 
Inputs:  Suburb, Rooms, Method, SellerG, Distance, Postcode, Bedroom2, Bathroom, Car, Landsize, BuildingArea, CouncilArea, Days.diff, Dist.bucket 
Number of observations:  6611 



In [20]:
write.table(impvars.re.h.imp, file = "F:\\melb.realstate.all.files\\OutputDataSets\\ImpVarsForPriceWithoutYear.h.csv",row.names=T, na="",col.names=T, sep=",")


In [21]:
## Applying Boruta for Type T ##
boruta.re.imp.t<-Boruta(Price~.,data = re.imp.t,doTrace=2)
print(boruta.re.imp.t)

 1. run of importance source...
 2. run of importance source...
 3. run of importance source...
 4. run of importance source...
 5. run of importance source...
 6. run of importance source...
 7. run of importance source...
 8. run of importance source...
 9. run of importance source...
 10. run of importance source...
 11. run of importance source...
After 11 iterations, +15 secs: 
 confirmed 12 attributes: Bathroom, Bedroom2, BuildingArea, Car, CouncilArea and 7 more;
 rejected 1 attribute: Months;
 still have 2 attributes left.

 12. run of importance source...
 13. run of importance source...
 14. run of importance source...
 15. run of importance source...
 16. run of importance source...
 17. run of importance source...
 18. run of importance source...
 19. run of importance source...
 20. run of importance source...
 21. run of importance source...
 22. run of importance source...
 23. run of importance source...
 24. run of importance source...
 25. run of importance source...


Boruta performed 99 iterations in 1.981737 mins.
 12 attributes confirmed important: Bathroom, Bedroom2, BuildingArea,
Car, CouncilArea and 7 more;
 2 attributes confirmed unimportant: Method, Months;
 1 tentative attributes left: Days.diff;


In [22]:
str(re.imp.t)

'data.frame':	679 obs. of  16 variables:
 $ Suburb      : Factor w/ 140 levels "Abbotsford","Aberfeldie",..: 3 3 3 3 3 3 6 7 7 7 ...
 $ Rooms       : Factor w/ 8 levels "1","2","3","4",..: 3 3 2 3 3 3 2 2 3 3 ...
 $ Price       : int  840000 700000 450000 700000 540000 825000 805000 720000 800000 705000 ...
 $ Method      : Factor w/ 5 levels "PI","S","SA",..: 1 2 2 1 4 4 2 2 4 2 ...
 $ SellerG     : Factor w/ 178 levels "Abercromby's",..: 111 21 13 21 111 13 106 73 73 73 ...
 $ Distance    : num  13.5 13.5 13.5 13.5 13.5 13.5 6.4 13.8 13.8 13.8 ...
 $ Postcode    : int  3042 3042 3042 3042 3042 3042 3078 3018 3018 3018 ...
 $ Bedroom2    : Factor w/ 11 levels "0","1","2","3",..: 4 4 3 4 3 4 3 3 4 4 ...
 $ Bathroom    : Factor w/ 9 levels "0","1","2","3",..: 3 3 2 3 3 3 3 2 3 3 ...
 $ Car         : Factor w/ 9 levels "0","1","2","3",..: 2 3 2 3 2 2 4 3 3 3 ...
 $ Landsize    : num  303 239 110 165 161 293 155 292 280 197 ...
 $ BuildingArea: num  225 134 90 133 101 102 140 125 139 152 

In [23]:
### As per Boruta Method and Months variables are not significant for T type, hence removing them ##
re.imp.t<-re.imp.t[,-c(4,16)]

In [24]:
## Cforest to know the significance of each individual variable on price with Type "t" ##
re.imp.t.rf<-party::cforest(Price~. ,data = re.imp.t, control=cforest_unbiased(mtry=13,ntree=200))
impvars.re.t.imp<-varImp(re.imp.t.rf)

In [25]:
print(impvars.re.t.imp)

                 Overall
Suburb       76503813423
Rooms         8022486291
SellerG       2467955362
Distance     10536475733
Postcode      6389283175
Bedroom2       373412757
Bathroom     15178460693
Car           1512444721
Landsize      3241205721
BuildingArea  6714920172
CouncilArea  16550771153
Days.diff      843111077
Dist.bucket   1740784212


In [27]:

write.table(impvars.re.t.imp, file = "F:\\melb.realstate.all.files\\OutputDataSets\\ImpVarsForPriceWithoutYear.t.csv",row.names=T, na="",col.names=T, sep=",")

In [28]:
## Applying Boruta for Type U ##
boruta.re.imp.u<-Boruta(Price~.,data = re.imp.u,doTrace=2)
print(boruta.re.imp.u)


 1. run of importance source...
 2. run of importance source...
 3. run of importance source...
 4. run of importance source...
 5. run of importance source...
 6. run of importance source...
 7. run of importance source...
 8. run of importance source...
 9. run of importance source...
 10. run of importance source...
 11. run of importance source...
After 11 iterations, +18 secs: 
 confirmed 13 attributes: Bathroom, Bedroom2, BuildingArea, Car, CouncilArea and 8 more;
 rejected 1 attribute: Months;
 still have 1 attribute left.

 12. run of importance source...
 13. run of importance source...
 14. run of importance source...
 15. run of importance source...
 16. run of importance source...
 17. run of importance source...
 18. run of importance source...
 19. run of importance source...
 20. run of importance source...
 21. run of importance source...
 22. run of importance source...
 23. run of importance source...
 24. run of importance source...
 25. run of importance source...
 

Boruta performed 99 iterations in 2.620561 mins.
 13 attributes confirmed important: Bathroom, Bedroom2, BuildingArea,
Car, CouncilArea and 8 more;
 1 attributes confirmed unimportant: Months;
 1 tentative attributes left: Days.diff;


In [29]:
### Deleting months as no it has no significance ###
re.imp.u<-re.imp.u[,-c(16)]

In [30]:
## Cforest to know the significance of each individual variable on price with Type "u" ##
re.imp.u.rf<-party::cforest(Price~. ,data = re.imp.u, control=cforest_unbiased(mtry=14,ntree=200))
impvars.re.u.imp<-varImp(re.imp.u.rf)

In [31]:
print(impvars.re.u.imp)

                 Overall
Suburb         664361662
Rooms        38003712048
Method        1239604033
SellerG       3328421410
Distance      7739692058
Postcode      5332008262
Bedroom2      1267094700
Bathroom      3605935633
Car            854216380
Landsize       943117507
BuildingArea   941531420
CouncilArea  14792503746
Days.diff      -42849884
Dist.bucket   2872526145


In [32]:
write.table(impvars.re.u.imp, file = "F:\\melb.realstate.all.files\\OutputDataSets\\ImpVarsForPriceWithoutYear.u.csv",row.names=T, na="",col.names=T, sep=",")
